In [1]:
# On the instance where you are running jupyter,
# authenticate with gcloud first:
#
#     gcloud auth application-default login

import bigframes

pd = bigframes.connect()

In [2]:
pd

In [3]:
df = pd.read_gbq("bigquery-public-data.chicago_taxi_trips.taxi_trips")

In [4]:
type(df)

bigframes.dataframe.DataFrame

In [5]:
df.head(n=1)

                                 unique_key  \
0  48b9324bea3f71b6ac1b8fc7c3005de0c044d441   

                                             taxi_id  \
0  2bbfe2af3f1b25128384590d49c415677e747671d8ea12...   

       trip_start_timestamp        trip_end_timestamp  trip_seconds  \
0 2018-02-28 10:30:00+00:00 2018-02-28 10:30:00+00:00           360   

   trip_miles  pickup_census_tract  dropoff_census_tract  \
0         0.8                 <NA>                  <NA>   

   pickup_community_area  dropoff_community_area  ...  extras  trip_total  \
0                   <NA>                    <NA>  ...     0.0         7.0   

   payment_type                    company  pickup_latitude pickup_longitude  \
0   Credit Card  Star North Management LLC              NaN              NaN   

  pickup_location  dropoff_latitude  dropoff_longitude dropoff_location  
0            None               NaN                NaN             None  

[1 rows x 23 columns]

In [6]:
df = df[[
    "company",
    "trip_miles",
    "fare",
    "tips",
]]

In [7]:
df

                        company  trip_miles   fare   tips
0                  Norshore Cab        19.0   0.01   0.00
1                  Norshore Cab        26.5   0.01   0.00
2                  Norshore Cab        14.3  40.80  10.20
3                  Norshore Cab         2.0   8.40   0.00
4                    Setare Inc         2.0   9.75   0.00
5       Chicago Elite Cab Corp.         0.0  11.65   1.00
6       Chicago Elite Cab Corp.         0.0   8.25   1.65
7       Chicago Elite Cab Corp.         0.0   6.65   1.33
...

[205574533 rows x 4 columns]

In [8]:
df['fare'].head(n=10)

0    10.00
1     5.75
2     7.50
3    46.00
4     8.00
5    22.17
6    34.00
7    45.00
8     6.25
9     3.25
Name: fare, dtype: float64

In [9]:
df['fare'].mean()

13.553347907200132

In [10]:
# Peek at the SQL.
print(df['fare'].mean()._value.compile())

SELECT avg(t0.`fare`) AS `tmp`
FROM (
  SELECT t1.`fare`
  FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips` t1
) t0


In [11]:
subtotal = df['fare'] + df['tips']
subtotal.head()

0    10.00
1     8.75
2     7.50
3    66.00
4     8.00
Name: fare, dtype: float64

In [12]:
subtotal[df['company'] == 'Norshore Cab'].sum()

2844629.760000093

In [13]:
# Peek at the SQL.
print(subtotal[df['company'] == 'Norshore Cab'].sum()._value.compile())

SELECT sum(t0.`fare`) AS `tmp`
FROM (
  SELECT CASE WHEN TRUE THEN t1.`fare` ELSE NULL END + CASE WHEN TRUE THEN t1.`tips` ELSE NULL END AS `fare`
  FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips` t1
  WHERE IFNULL(t1.`company` = 'Norshore Cab', FALSE)
) t0
